In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

import pickle

In [3]:
with open('finalknnmodel.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
public_test = pd.read_csv("data/test_public.csv")
data = pd.read_csv("data/train.csv")

In [5]:
training = data[~data['MISSING_DATA']]

# adding date time, modified from given feature engineering notebook
getdate = lambda x : [tuple([i.year, i.month, i.day, i.hour, i.weekday()]) for i in [datetime.fromtimestamp(x["TIMESTAMP"])]][0]

training[["YR", "MON", "DAY", "HR", "WK"]] = training[["TIMESTAMP"]].apply(getdate, axis=1, result_type="expand")
training = training.drop(columns = ['TIMESTAMP'])

public_test[["YR", "MON", "DAY", "HR", "WK"]] = public_test[["TIMESTAMP"]].apply(getdate, axis=1, result_type="expand")
public_test = public_test.drop(columns = ['TIMESTAMP'])

# adding trip time, using the given 15 second formula on polyline
training['TRIP_TIME'] = training['POLYLINE'].apply(lambda x : max(x.count(".")/2-1,0)*15)
training = training.drop(columns = ['POLYLINE'])

C:\Users\ishpa\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [6]:
days = [(8,14),(9,30),(10,6),(10,31),(12,21)]

narrows = [training[(training['MON'] == i[0])&(training['DAY']==i[1])].drop(columns = ['MON','DAY']).assign(date=j) for j,i in enumerate(days)]
tnarrows = [public_test[(public_test['MON'] == i[0])&(public_test['DAY']==i[1])].drop(columns = ['MON','DAY']).assign(date=j) for j,i in enumerate(days)]

narrowed = pd.concat(narrows)
tnarrowed = pd.concat(tnarrows)

In [7]:
def knntrans(data, test=pd.DataFrame(), w = [3,0,0,1,0]):
    data = pd.concat([data,test])
    data = data.drop(columns = ['TRIP_ID','DAY_TYPE','MISSING_DATA','YR'])
    ones = pd.get_dummies(data['CALL_TYPE'])*w[0]
    ones = ones.join(pd.get_dummies(data['ORIGIN_STAND'])*w[1])
    ones = ones.join(pd.get_dummies(data['TAXI_ID'])*w[2])
    ones = ones.join(pd.get_dummies(((pd.get_dummies(data['HR'])*w[3])).rename(columns = {i:f'h{i}' for i in range(24)})))
    ones = ones.join((pd.get_dummies(data['date'])*w[4]).rename(columns = {i:f'date{i}' for i in range(5)}))
    return ones.iloc[:-test.shape[0],:],ones.iloc[-test.shape[0]:,:]#.to_numpy()

In [8]:
pkt,pkpublic_test = knntrans(narrowed,tnarrowed)

In [9]:
preds = model.predict(pkpublic_test)

In [10]:
knnpred = pd.read_csv("data/sampleSubmission.csv")
no_125 = (tnarrowed[['TRIP_ID']])
no_125['TRAVEL_TIME'] = preds
knnpred = knnpred.merge(no_125, how = 'left', right_on = 'TRIP_ID', left_on = 'TRIP_ID')
knnpred = knnpred.drop(columns = 'TRAVEL_TIME_x').rename(columns = {'TRAVEL_TIME_y' : 'TRAVEL_TIME'})
knnpred['TRAVEL_TIME'][125] = 716

C:\Users\ishpa\AppData\Local\Temp\ipykernel_19444\3982142290.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_125['TRAVEL_TIME'] = preds
C:\Users\ishpa\AppData\Local\Temp\ipykernel_19444\3982142290.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  knnpred['TRAVEL_TIME'][125] = 716


## The predictions are now in knnpred dataframe displayed below, uncomment the last cell to turn to csv

In [11]:
knnpred

,TRIP_ID,TRAVEL_TIME
0,T1,675.0
1,T2,675.0
2,T3,675.0
3,T4,675.0
4,T5,675.0
...,...,...
315,T323,783.0
316,T324,786.0
317,T325,1285.5
318,T326,783.0


In [12]:
# knnpred.to_csv("knn_final_pred.csv", index=None)